In [1]:
name="mimikkyu"

デフォルトマーカー位置のインプット

In [2]:
import numpy as np 
import cv2
import glob
import os

aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
dst=cv2.imread( "./img/"+"/md_"+str(name)+".png") 
def_corners, def_ids, rejectedImgPoints = aruco.detectMarkers(dst, dictionary)

count=[0,1,2,3]
count2=[0,1,2,3,4,5,6,7]

def_pts=np.zeros((8,4,2),dtype = np.float32)

for i in count2:
    for j in count2:
        if j ==def_ids[i][0]:
            for k in count:
                def_pts[j][k][0]=def_corners[i][0][k][0]
                def_pts[j][k][1]=def_corners[i][0][k][1]
 
        
height, width, color = dst.shape


def_bord_pts=np.zeros((16,2),dtype = np.float32)
def_pro_pts=np.zeros((16,2),dtype = np.float32)

count=[0,1,2,3]
def_pts
for i in count:
    for j in count:
        def_bord_pts[4*i+j][0]=def_pts[i][j][0]
        def_bord_pts[4*i+j][1]=def_pts[i][j][1]    

for i in count:
    for j in count:
        def_pro_pts[4*i+j][0]=def_pts[4+i][j][0]
        def_pro_pts[4*i+j][1]=def_pts[4+i][j][1]


print("def_pts")
print(def_pts)
print("def_pro_pts")
print(def_pro_pts)
print("def_bord_pts")
print(def_bord_pts)

def_pts
[[[  5.   7.]
  [ 34.   7.]
  [ 34.  36.]
  [  5.  36.]]

 [[365.   7.]
  [394.   7.]
  [394.  36.]
  [365.  36.]]

 [[  5. 187.]
  [ 34. 187.]
  [ 34. 216.]
  [  5. 216.]]

 [[365. 187.]
  [394. 187.]
  [394. 216.]
  [365. 216.]]

 [[185.   7.]
  [214.   7.]
  [214.  36.]
  [185.  36.]]

 [[365.  97.]
  [394.  97.]
  [394. 126.]
  [365. 126.]]

 [[185. 187.]
  [214. 187.]
  [214. 216.]
  [185. 216.]]

 [[  5.  97.]
  [ 34.  97.]
  [ 34. 126.]
  [  5. 126.]]]
def_pro_pts
[[185.   7.]
 [214.   7.]
 [214.  36.]
 [185.  36.]
 [365.  97.]
 [394.  97.]
 [394. 126.]
 [365. 126.]
 [185. 187.]
 [214. 187.]
 [214. 216.]
 [185. 216.]
 [  5.  97.]
 [ 34.  97.]
 [ 34. 126.]
 [  5. 126.]]
def_bord_pts
[[  5.   7.]
 [ 34.   7.]
 [ 34.  36.]
 [  5.  36.]
 [365.   7.]
 [394.   7.]
 [394.  36.]
 [365.  36.]
 [  5. 187.]
 [ 34. 187.]
 [ 34. 216.]
 [  5. 216.]
 [365. 187.]
 [394. 187.]
 [394. 216.]
 [365. 216.]]


貼り付け関数

In [3]:
def cvpaste(img, imgback, x, y, angle, scale):  
    # x and y are the distance from the center of the background image 

    r = img.shape[0]
    c = img.shape[1]
    rb = imgback.shape[0]
    cb = imgback.shape[1]
    hrb=round(rb/2)
    hcb=round(cb/2)
    hr=round(r/2)
    hc=round(c/2)

    # Copy the forward image and move to the center of the background image
    imgrot = np.zeros((rb,cb,3),np.uint8)
    imgrot[hrb-hr:hrb+hr,hcb-hc:hcb+hc,:] = img[:hr*2,:hc*2,:]

    # Rotation and scaling
    M = cv2.getRotationMatrix2D((hcb,hrb),angle,scale)
    imgrot = cv2.warpAffine(imgrot,M,(cb,rb))
    # Translation
    M = np.float32([[1,0,x],[0,1,y]])
    imgrot = cv2.warpAffine(imgrot,M,(cb,rb))

    # Making mask
    imggray = cv2.cvtColor(imgrot,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(imggray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Now black-out the area of the forward image in the background image
    img1_bg = cv2.bitwise_and(imgback,imgback,mask = mask_inv)

    # Take only region of the forward image.
    img2_fg = cv2.bitwise_and(imgrot,imgrot,mask = mask)

    # Paste the forward image on the background image
    imgpaste = cv2.add(img1_bg,img2_fg)

    return imgpaste


プロジェクタ起動

In [7]:
import cv2
import numpy as np
import time

aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
img=cv2.imread("img/"+str(name)+".jpg")
imgback = cv2.imread("img/"+str(name)+".jpg")

height,width,color=imgback.shape

angle=0
scale=0.2

ar4=cv2.imread("ARUCO/blue_ar04.png")
ar5=cv2.imread("ARUCO/blue_ar05.png")
ar6=cv2.imread("ARUCO/blue_ar06.png")
ar7=cv2.imread("ARUCO/blue_ar07.png")

cap = cv2.VideoCapture(1)

rt_pts=np.zeros((8,4,2),dtype = np.float32)
rt_bord_pts=np.zeros((16,2),dtype = np.float32)
rt_pro_pts=np.zeros((16,2),dtype = np.float32)

count=[0,1,2,3]
count2=[0,1,2,3,4,5,6,7]

mat1, mask1 = cv2.findHomography(def_pro_pts, def_pro_pts, cv2.RANSAC,1)#初期設定（変更なし）
mat2, mask2 = cv2.findHomography(def_bord_pts, def_bord_pts, cv2.RANSAC,1)

while(True):
    
    t0 = time.time()    
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    frame2=cv2.warpPerspective(img, mat2, (3*width, 3*height))
    frame3=cv2.warpPerspective(frame2, mat1, (width, height))#カメラから見たプロジェクタ上の座標→カメラから見たボード上の座標
    

    imgpaste = cvpaste(ar4, frame3, 0, -90, angle, scale)
    imgpaste2 = cvpaste(ar5, imgpaste, 180, 0, angle, scale)
    imgpaste3 = cvpaste(ar6, imgpaste2, 0, 90, angle, scale)
    imgpaste4 = cvpaste(ar7, imgpaste3, -180, 0, angle, scale)

    
    cv2.namedWindow('screen', cv2.WINDOW_NORMAL)
    cv2.setWindowProperty('screen', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('screen',imgpaste4)
    
    new_corners, new_ids, rejectedImgPoints = aruco.detectMarkers(frame, dictionary)
    #print(rt_pro_pts)
    #print(rejectedImgPoints)
    if not new_corners:
        pass
        
    else:
        num=new_ids.size-1
    
        if num==7:#全部読み込まれてるならば
            for i in count2:
                for j in count2:
                    if j ==new_ids[i][0]:
                        for h in count:
                            rt_pts[j][h][0]=new_corners[i][0][h][0]
                            rt_pts[j][h][1]=new_corners[i][0][h][1]
                if num==i:
                    break
                    
            for i in count:
                for j in count:
                    rt_bord_pts[4*i+j][0]=rt_pts[i][j][0]
                    rt_bord_pts[4*i+j][1]=rt_pts[i][j][1]

            for i in count:
                for j in count:
                    rt_pro_pts[4*i+j][0]=rt_pts[4+i][j][0]
                    rt_pro_pts[4*i+j][1]=rt_pts[4+i][j][1]

                
        
            mat1, mask1 = cv2.findHomography(rt_pro_pts, def_pro_pts, cv2.RANSAC,1)
            mat2, mask2 = cv2.findHomography(def_bord_pts, rt_bord_pts, cv2.RANSAC,1)
    #print(cap.get(cv2.CAP_PROP_FPS))

 #   print(rt_pro_pts)
    if cv2.waitKey(1) &0xFF == ord('q'):
        break
    
    t1 = time.time()                # 計測終了時間
    elapsed_time = float(t1 - t0)   # 経過時間
   # print(elapsed_time)             # 経過時間を表示
    
cv2.destroyAllWindows()

In [11]:
def_bord_pts

array([[  5.,   7.],
       [ 34.,   7.],
       [ 34.,  36.],
       [  5.,  36.],
       [365.,   7.],
       [394.,   7.],
       [394.,  36.],
       [365.,  36.],
       [  5., 187.],
       [ 34., 187.],
       [ 34., 216.],
       [  5., 216.],
       [365., 187.],
       [394., 187.],
       [394., 216.],
       [365., 216.]], dtype=float32)

In [15]:
new_corners

[array([[[420., 349.],
         [481., 355.],
         [483., 413.],
         [423., 407.]]], dtype=float32),
 array([[[102., 270.],
         [137., 276.],
         [135., 316.],
         [100., 310.]]], dtype=float32),
 array([[[ 80.,  89.],
         [115.,  85.],
         [116., 128.],
         [ 80., 132.]]], dtype=float32),
 array([[[431.,  39.],
         [484.,  39.],
         [478.,  94.],
         [426.,  92.]]], dtype=float32)]

In [54]:
import cv2
from datetime import datetime
img=cv2.imread("img/"+"/md_"+str(name)+".png")

cap = cv2.VideoCapture(1)

while(True):
    cv2.namedWindow('screen', cv2.WINDOW_NORMAL)
    cv2.setWindowProperty('screen', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('screen', img)
    
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)
   
    k = cv2.waitKey(1)&0xff # キー入力を待つ
    if k == ord('p'):
        # 「p」キーで画像を保存
        date = datetime.now().strftime("%Y%m%d_%H%M%S")
        path = "./projector_img/" + date + ".png"
        cv2.imwrite(path, frame) # ファイル保存

        cv2.imshow(path, frame) # キャプチャした画像を表示
    elif k == ord('q'):
        # 「q」キーが押されたら終了する
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [55]:
import numpy as np 
import cv2
import glob
import os

aruco = cv2.aruco

dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

image_dir = './projector_img/'
search_pattern = '*.png'
datas = []
for image_path in glob.glob(os.path.join(image_dir,search_pattern)):
    # (height,width,channels)
    data = cv2.imread(image_path)
    # (1,height,width,channels)
    data_expanded = np.expand_dims(data,axis=0)
    datas.append(data_expanded)
# (n_samples,height,width,channels)
image_datas = np.concatenate(datas,axis=0)

p_img = image_datas[0]
pro_corners, pro_ids, rejectedImgPoints = aruco.detectMarkers(p_img, dictionary)
pt1=[pro_corners[0][0][0][0],pro_corners[0][0][0][1],pro_ids[0][0]]

count=[0,1,2,3]

j=pro_ids.size-1

rt_pts=np.zeros((4,4,2),dtype = np.float32)

for i in count:
    for k in count:
        if k ==pro_ids[i][0]:
            for h in count:
                rt_pts[k][h][0]=pro_corners[i][0][h][0]
                rt_pts[k][h][1]=pro_corners[i][0][h][1]
    if j==i:
        break
rt_pts

array([[[ 41., 139.],
        [ 80., 138.],
        [ 77., 177.],
        [ 37., 177.]],

       [[516., 125.],
        [558., 123.],
        [561., 163.],
        [519., 164.]],

       [[ 23., 373.],
        [ 65., 372.],
        [ 62., 416.],
        [ 20., 417.]],

       [[532., 362.],
        [576., 362.],
        [579., 407.],
        [534., 407.]]], dtype=float32)

In [56]:
import cv2
from datetime import datetime

cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_EXPOSURE, 40)

while(True):
   
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)
   
    k = cv2.waitKey(1)&0xff # キー入力を待つ
    if k == ord('p'):
        # 「p」キーで画像を保存
        date = datetime.now().strftime("%Y%m%d_%H%M%S")
        path = "./bord_img/" + date + ".png"
        cv2.imwrite(path, frame) # ファイル保存

        cv2.imshow(path, frame) # キャプチャした画像を表示
    elif k == ord('q'):
        # 「q」キーが押されたら終了する
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [57]:
import numpy as np 
import cv2
import glob
import os

aruco = cv2.aruco

dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

image_dir = './bord_img/'
search_pattern = '*.png'
datas = []
for image_path in glob.glob(os.path.join(image_dir,search_pattern)):
    # (height,width,channels)
    data = cv2.imread(image_path)
    # (1,height,width,channels)
    data_expanded = np.expand_dims(data,axis=0)
    datas.append(data_expanded)
# (n_samples,height,width,channels)
image_datas = np.concatenate(datas,axis=0)

b_img = image_datas[0]
bor_corners, bor_ids, rejectedImgPoints = aruco.detectMarkers(b_img, dictionary)

j=bor_ids.size-1
count=[0,1,2,3]

rt_pts2=np.zeros((4,4,2),dtype = np.float32)

for i in count:
    for k in count:
        if k ==bor_ids[i][0]:
            for h in count:
                rt_pts2[k][h][0]=bor_corners[i][0][h][0]
                rt_pts2[k][h][1]=bor_corners[i][0][h][1]
    if j==i:
        break
rt_pts2

array([[[ 34., 119.],
        [ 79., 121.],
        [ 76., 163.],
        [ 30., 162.]],

       [[538., 129.],
        [579., 130.],
        [582., 170.],
        [540., 170.]],

       [[ 15., 379.],
        [ 63., 379.],
        [ 60., 428.],
        [ 10., 429.]],

       [[554., 375.],
        [597., 374.],
        [601., 420.],
        [558., 421.]]], dtype=float32)

In [66]:
import cv2
import numpy as np
def_det_pts=np.zeros((16,2),dtype = np.float32)
rt_det_pts=np.zeros((16,2),dtype = np.float32)
rt_det_pts2=np.zeros((16,2),dtype = np.float32)

count=[0,1,2,3]
k=0
for i in count:
    if rt_pts[i][0][0]==rt_pts[i][0][1]==0 or rt_pts2[i][0][0]==rt_pts2[i][0][1]==0:
        print(str(i)+"番のARは検出できませんでした")
        for j in count2:    
            def_det_pts=np.delete(def_det_pts,-1,0)
            rt_det_pts=np.delete(rt_det_pts,-1,0)
            rt_det_pts2=np.delete(rt_det_pts2,-1,0)
    
    else:
        for j in count:
            def_det_pts[4*k+j][0]=def_pts[i][j][0]
            def_det_pts[4*k+j][1]=def_pts[i][j][1]
        
            rt_det_pts[4*k+j][0]=rt_pts[i][j][0]
            rt_det_pts[4*k+j][1]=rt_pts[i][j][1]

            rt_det_pts2[4*k+j][0]=rt_pts2[i][j][0]
            rt_det_pts2[4*k+j][1]=rt_pts2[i][j][1]
   
        k=k+1  

mat1, mask1 = cv2.findHomography(rt_det_pts2,def_det_pts, cv2.RANSAC,1)#カメラ座標からボード座標への変換
img2=cv2.warpPerspective(p_img, mat1, (width, height))#ボード座標系におけるプロジェクタ投影画像の座標
img2_corners, img2_ids, rejectedImgPoints = aruco.detectMarkers(img2, dictionary)

j=img2_ids.size-1
count=[0,1,2,3]

img2_pts=np.zeros((4,4,2),dtype = np.float32)

for i in count:
    for k in count:
        if k ==img2_ids[i][0]:
            for h in count:
                img2_pts[k][h][0]=img2_corners[i][0][h][0]
                img2_pts[k][h][1]=img2_corners[i][0][h][1]
    if j==i:
        break
        



画像確認（ボード座標系に変換した投影画像）

In [67]:
while(True):
    cv2.imshow('frame', img2)
    k = cv2.waitKey(1)&0xff # キー入力を待つ
    if k == ord('q'):
        break
cv2.destroyAllWindows()

プロジェクタ位置変更

In [140]:
import cv2
import numpy as np
from datetime import datetime
from pathlib import Path


aruco = cv2.aruco

dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# 画像ファイルのパス
or_img=cv2.imread("img/"+str(name)+".jpg")
h,w,c=or_img.shape
frame1 = cv2.imread("img/"+str(name)+".jpg")


cap = cv2.VideoCapture(1)
#cap.set(cv2.CAP_PROP_EXPOSURE, 400)
while(True):
   
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    k = cv2.waitKey(1)&0xff # キー入力を待つ
    if k == ord('p'):
        # 「p」キーで画像を保存
        path = "./rt_img/realtime.png"
        cv2.imwrite(path, frame) # ファイル保存
        cv2.imshow(path, frame) # キャプチャした画像を表示
    elif k == ord('q'):
        # 「q」キーが押されたら終了する
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()



新しいボード座標

In [141]:
rt_img = cv2.imread("rt_img/realtime.png")
new_corners, new_ids, rejectedImgPoints = aruco.detectMarkers(rt_img, dictionary)

j=new_ids.size-1
count=[0,1,2,3]

rt_pts3=np.zeros((4,4,2),dtype = np.float32)

for i in count:
    for k in count:
        if k ==new_ids[i][0]:
            for h in count:
                rt_pts3[k][h][0]=new_corners[i][0][h][0]
                rt_pts3[k][h][1]=new_corners[i][0][h][1]
    if j==i:
        break
    
def_det_pts=np.zeros((16,2),dtype = np.float32)
rt_det_pts3=np.zeros((16,2),dtype = np.float32)
rt_det_pts2=np.zeros((16,2),dtype = np.float32)
rt_det_pts=np.zeros((16,2),dtype = np.float32)
img2_det_pts=np.zeros((16,2),dtype = np.float32)#Pb


count=[0,1,2,3]
k=0
for i in count:
    if rt_pts3[i][0][0]==rt_pts3[i][0][1]==0 or rt_pts2[i][0][0]==rt_pts2[i][0][1]==0 or img2_pts[i][0][0]==img2_pts[i][0][1]==0 or rt_pts[i][0][0]==rt_pts[i][0][1]==0:
        print(str(i)+"番のARは検出できませんでした")
        for j in count:    
            def_det_pts=np.delete(def_det_pts,-1,0)
            rt_det_pts=np.delete(rt_det_pts,-1,0)
            rt_det_pts2=np.delete(rt_det_pts2,-1,0)
            rt_det_pts3=np.delete(rt_det_pts3,-1,0)
            img2_det_pts=np.delete(img2_det_pts,-1,0)
    
    else:
        for j in count:
            def_det_pts[4*k+j][0]=def_pts[i][j][0]
            def_det_pts[4*k+j][1]=def_pts[i][j][1]
        
            rt_det_pts3[4*k+j][0]=rt_pts3[i][j][0]
            rt_det_pts3[4*k+j][1]=rt_pts3[i][j][1]

            rt_det_pts2[4*k+j][0]=rt_pts2[i][j][0]
            rt_det_pts2[4*k+j][1]=rt_pts2[i][j][1]
            
            rt_det_pts[4*k+j][0]=rt_pts[i][j][0]
            rt_det_pts[4*k+j][1]=rt_pts[i][j][1]

            img2_det_pts[4*k+j][0]=img2_pts[i][j][0]
            img2_det_pts[4*k+j][1]=img2_pts[i][j][1]
   
        k=k+1  

変換行列生成

In [142]:
mat2, mask2 = cv2.findHomography(img2_det_pts, def_det_pts, cv2.RANSAC,1)#Hp
mat3, mask3 = cv2.findHomography(rt_det_pts, img2_det_pts, cv2.RANSAC,1)#Hc-1
mat4, mask4 = cv2.findHomography(def_det_pts, rt_det_pts3, cv2.RANSAC,1)


出力

In [143]:
or_img=cv2.imread("img/"+str(name)+".jpg")
height,width,color=or_img.shape

while(True):

    frame2=cv2.warpPerspective(or_img, mat4, (width*2, height*2))#
    frame3=cv2.warpPerspective(frame2, mat2 ,(width*2, height*2))
    frame4=cv2.warpPerspective(frame3, mat3 ,(width, height))
   
   
    cv2.namedWindow('screen', cv2.WINDOW_NORMAL)
    cv2.setWindowProperty('screen', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('screen',frame4)
        
    if cv2.waitKey(10) &0xFF == ord('q'):
        break

  

cv2.destroyAllWindows()

In [129]:
def_det_pts

array([[ 14.,  14.],
       [ 43.,  14.],
       [ 43.,  43.],
       [ 14.,  43.],
       [356.,  14.],
       [385.,  14.],
       [385.,  43.],
       [356.,  43.],
       [356., 181.],
       [385., 181.],
       [385., 210.],
       [356., 210.]], dtype=float32)

In [1]:
rt_det_pts

NameError: name 'rt_det_pts' is not defined

In [103]:
rt_det_pts2

array([[ 34., 119.],
       [ 79., 121.],
       [ 76., 163.],
       [ 30., 162.],
       [ 15., 379.],
       [ 63., 379.],
       [ 60., 428.],
       [ 10., 429.]], dtype=float32)

In [104]:
img2_det_pts

array([[ 19.,  28.],
       [ 44.,  26.],
       [ 45.,  52.],
       [ 20.,  54.],
       [ 19., 177.],
       [ 44., 177.],
       [ 44., 203.],
       [ 19., 203.]], dtype=float32)